In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder ,OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]

In [3]:
#preprocess the data
#drop irrelevant colums
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France  Female  ...               1        101348.88       1
1             608     Spain  Female  ...               1        112542.58       0
2             502    France  Female  ...               0        113931.57       1
3             699    France  Female  ...               0         93826.63       0
4             850     Spain  Female  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France    Male  ...               0         96270.64       0
9996          516    France    Male  ...               1        101699.77       0
9997          709    France  Female  ...               1         42085.58       1
9998          772   Germany    Male  ...               0         92888.52       1
9999          792    France  Female  ...               0         38190.78       0

[10000 rows x 11 columns]

In [5]:
##encoding categorical variables
Label_encoder_gender=LabelEncoder()
data['Gender']=Label_encoder_gender.fit_transform(data['Gender'])
data

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0             619    France       0  ...               1        101348.88       1
1             608     Spain       0  ...               1        112542.58       0
2             502    France       0  ...               0        113931.57       1
3             699    France       0  ...               0         93826.63       0
4             850     Spain       0  ...               1         79084.10       0
...           ...       ...     ...  ...             ...              ...     ...
9995          771    France       1  ...               0         96270.64       0
9996          516    France       1  ...               1        101699.77       0
9997          709    France       0  ...               1         42085.58       1
9998          772   Germany       1  ...               0         92888.52       1
9999          792    France       0  ...               0         38190.78       0

[10000 rows x 11 columns]

In [7]:
##one hot encoding georgrapy
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [8]:
#combine onehot encoded columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0          619       0  ...                0.0              0.0
1          608       0  ...                0.0              1.0
2          502       0  ...                0.0              0.0
3          699       0  ...                0.0              0.0
4          850       0  ...                0.0              1.0

[5 rows x 13 columns]

In [9]:
#split the data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [11]:
##save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(Label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)
    

__ANN Regression Problem Statement__|

In [12]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [14]:
#build the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)##default activation-- Linear activation function
])
#compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
##setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [18]:
##setup early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [19]:
#Train the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100292.6641 - mae: 100292.6641 - val_loss: 98553.6562 - val_mae: 98553.6562
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 100069.7891 - mae: 100069.7891 - val_loss: 97267.3516 - val_mae: 97267.3516
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98587.1797 - mae: 98587.1797 - val_loss: 93919.6094 - val_mae: 93919.6094
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 94719.0781 - mae: 94719.0781 - val_loss: 88161.6797 - val_mae: 88161.6797
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 87620.4141 - mae: 87620.4141 - val_loss: 80365.5703 - val_mae: 80365.5703
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 79628.2422 - mae: 79628.2422 - val_loss: 71749.9531 - val_mae: 71749.9531
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 70811.2188 - mae: 70811.2188 - val_loss: 63892.4141 - val_mae: 63892.4141
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/ste

In [20]:
%load_ext tensorboard 

In [22]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 11512), started 0:00:56 ago. (Use '!kill 11512' to kill it.)

In [23]:
##evaluate model on the test date
test_loss,test_mae= model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51061.5820 - mae: 51061.5820
Test MAE : 50283.67578125


In [24]:
model.save('regression_model.h5')